In [7]:
# Install LightGBM if not already installed
!pip install lightgbm

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.decomposition import PCA  # Added PCA
from imblearn.over_sampling import ADASYN
import lightgbm as lgb
import optuna
import joblib
from sklearn.metrics import accuracy_score

# Load the dataset
data = pd.read_csv('final_dataset_10000.csv')

# Drop unnecessary columns
data = data.drop(['Conversion_Rate', 'Target_Audience'], axis=1)

# Convert 'Date' to datetime and extract useful features
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

# Drop the original 'Date' column
data = data.drop('Date', axis=1)

# Extract numeric part from the 'Duration' column and convert to integer
data['Duration'] = data['Duration'].str.extract('(\d+)').astype(int)

# Feature Engineering: Add interaction features
data['CTR'] = data['Clicks'] / data['Impressions']

# Log transform skewed numeric features using PowerTransformer
power = PowerTransformer(method='yeo-johnson', standardize=True)
data[['Ad_Budget', 'Impressions']] = power.fit_transform(data[['Ad_Budget', 'Impressions']])

# Encode categorical variables using One-Hot Encoding
data = pd.get_dummies(data, columns=['Ad_Type'], drop_first=True)

# Define features and target
X = data.drop('Campaign_Type', axis=1)
y = data['Campaign_Type']

# Encode target variable
y_encoded = pd.factorize(y)[0]

# Apply ADASYN for data balancing
adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X, y_encoded)

# Scale numeric features
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)

# Apply PCA for feature reduction
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_resampled_pca = pca.fit_transform(X_resampled_scaled)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled_pca, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled)

# Define the objective function for Optuna
def objective(trial):
    params = {
        'objective': 'multiclass',
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
        'num_class': len(np.unique(y_resampled)),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return accuracy_score(y_test, y_pred)

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Increase n_trials for better results

# Get the best parameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)

# Train the model with the best parameters
best_lgb = lgb.LGBMClassifier(**best_params, random_state=42)
best_lgb.fit(X_train, y_train)

# Calculate Train Accuracy
y_train_pred = best_lgb.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_accuracy)

# Make predictions
y_pred = best_lgb.predict(X_test)

# Print classification report and confusion matrix
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Save the model
joblib.dump(best_lgb, 'lightgbm_optuna_tuned_pca.pkl')

[I 2025-03-13 12:57:54,673] A new study created in memory with name: no-name-10502e87-f0b8-4d49-9c0d-52934ae51706
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002420 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2805
[LightGBM] [Info] Number of data points in the train set: 11102, number of used features: 11
[LightGBM] [Info] Start training from score -1.403923
[LightGBM] [Info] Start training from score -1.376435
[LightGBM] [Info] Start training from score -1.382160
[LightGBM] [Info] Start training from score -1.382877
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-13 12:57:56,520] Trial 0 finished with value: 0.3432114333753678 and parameters: {'num_leaves': 242, 'learning_rate': 0.23275916594687487, 'max_depth': 5, 'min_child_samples': 44, 'subsample': 0.5760351562241357, 'colsample_bytree': 0.7843099118723389, 'reg_alpha': 0.93765762943175, 'reg_lambda': 0.44778559417827146}. Best is trial 0 with value: 0.3432114333753678.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002343 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2805
[LightGBM] [Info] Number of data points in the train set: 11102, number of used features: 11
[LightGBM] [Info] Start training from score -1.403923
[LightGBM] [Info] Start training from score -1.376435
[LightGBM] [Info] Start training from score -1.382160
[LightGBM] [Info] Start training from score -1.382877
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
[I 2025-03-13 12:58:00,042] Trial 1 finished with value: 0.3604455653635982 and parameters: {'num_leaves': 193, 'learning_rate': 0.21702242869237362, 'max_depth': 8, 'min_child_samples': 45, 'subsample': 0.6498895134408987, 'colsample_bytree': 0.8860236870660574, 'reg_alpha': 0.2960301568645224, 'reg_lambda': 0.34635048887486797}. Best is trial 1 with value: 0.3604455653635982.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Train Accuracy: 0.9933345343181409


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Classification Report:
              precision    recall  f1-score   support

           0       0.40      0.34      0.36      1168
           1       0.37      0.51      0.43      1201
           2       0.40      0.35      0.37      1195
           3       0.37      0.34      0.35      1194

    accuracy                           0.38      4758
   macro avg       0.38      0.38      0.38      4758
weighted avg       0.38      0.38      0.38      4758

Confusion Matrix:
[[393 331 211 233]
 [186 611 186 218]
 [203 335 413 244]
 [212 361 221 400]]


['lightgbm_optuna_tuned_pca.pkl']